# Chatbot Project

To do's:
- convert it to csv 

## Step 1: Upload and read documents

Installations:
pip install python-docx

In [6]:
#IMPORTS
from docx import Document
import pandas as pd

In [11]:
'''
#This is if the load in is a docx, I could not figure out how to deal with the paragraph thing

# Specify the path to your .docx file
#"C:\\Users\\anett\\NewFacultyOrientationResearchFAQs.docx"
#"C:\\Users\\anett\\CopyofNewFacultyOrientationResearch.docx"
docx_file_path = 'C:\\Users\\anett\\CopyofNewFacultyOrientationResearch.docx'

# Open the .docx file
doc = Document(docx_file_path)

# Iterate through paragraphs and print the text
for paragraph in doc.paragraphs:
        print(paragraph.text)
'''

'\n#This is if the load in is a docx, I could not figure out how to deal with the paragraph thing\n\n# Specify the path to your .docx file\n#"C:\\Users\\anett\\NewFacultyOrientationResearchFAQs.docx"\n#"C:\\Users\\anett\\CopyofNewFacultyOrientationResearch.docx"\ndocx_file_path = \'C:\\Users\\anett\\CopyofNewFacultyOrientationResearch.docx\'\n\n# Open the .docx file\ndoc = Document(docx_file_path)\n\n# Iterate through paragraphs and print the text\nfor paragraph in doc.paragraphs:\n        print(paragraph.text)\n'

In [13]:
#We are gonna try to convert a docx file to a .txt for easier coding

docx_file_path = 'C:\\Users\\anett\\CopyofNewFacultyOrientationResearch.docx'
doc = Document(docx_file_path)

# Create or open a text file to write the extracted text
txt_file_path = 'EditedNewFacultyOrientationResearch.txt'
with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
    # Iterate through paragraphs and write the text to the text file
    for paragraph in doc.paragraphs:
        txt_file.write(paragraph.text + '\n')

print(f'Text extracted from {docx_file_path} and saved to {txt_file_path}')

Text extracted from C:\Users\anett\CopyofNewFacultyOrientationResearch.docx and saved to EditedNewFacultyOrientationResearch.txt


In [19]:
'''
So for this I made an edited doc. This is since we can request the 
ways docs are formatted

The format is:
Question
Answer
Tag --> if dont know just put unknown
<Space>

currently allowing for them to do this in word doc and save as docx since
 we are allowing for google API I thought it was fitting.
This also allows for the CS team to connect with the DB team and be able
to edit and delete one and a time. We will connect them with this code if they 
ever choose to upload a file and this will be run. 
'''

txt_file_path = 'C:\\Users\\anett\\EditedNewFacultyOrientationResearch.txt'
questions = []
answers = []
tags = []

# Read the text file and iterate through non-empty lines
with open(txt_file_path, 'r', encoding='utf-8') as txt_file:
    lines = [line.strip() for line in txt_file.readlines() if line.strip()]
    
    # Iterate through lines in sets of 3
    for i in range(0, len(lines), 3):
        # Check if there are enough lines left to extract a set
        if i + 2 < len(lines):
            questions.append(lines[i])
            answers.append(lines[i + 1])
            tags.append(lines[i + 2])
        else:
            # Handle the case where there are not enough lines for a complete set
            print(f"Skipping incomplete set at index {i}.")

# Create a DataFrame from the lists of questions, answers, and tags
df = pd.DataFrame({'Question': questions, 'Answer': answers, 'Tag': tags})

# Display the first few rows of the DataFrame
df.head()

,Question,Answer,Tag
0,"I would like to submit a proposal, how do I ge...",Please start a routing sheet (SP proposal) in ...,proposal
1,How soon should I start my Cayuse routing sheet?,"Please start the first page, General tab, of t...",other
2,What are IIT’s internal deadlines?,The administrative portions of the proposal (b...,other
3,How will I know who my assigned research admin...,"After you start your routing sheet, OSRP will ...",research
4,OSRP only handles federally sponsored projects...,No. OSRP handles all sponsored projects for th...,other


## Step 2: Load the dataframe into mySQL

In [ ]:
#Installations which may be necessary
#pip install pandas sqlalchemy pymysql

In [ ]:
#Imports
#from sqlalchemy import create_engine

In [ ]:
'''
# Replace these variables with your MySQL database credentials
db_username = 'your_username'
db_password = 'your_password'
db_host = 'your_host'
db_port = 'your_port'
db_name = 'your_database_name'

# Create a connection string for SQLAlchemy
connection_string = f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Replace df with your DataFrame
df = pd.DataFrame({
    'Question': ['Q1', 'Q2', 'Q3'],
    'Answer': ['A1', 'A2', 'A3'],
    'Tag': ['T1', 'T2', 'T3']
})

# Replace 'your_table_name' with the desired table name
table_name = 'your_table_name'

# Write the DataFrame to MySQL
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print(f'DataFrame successfully written to MySQL table: {table_name}')
'''


## Step 3: Grab user question and give it filters

In [24]:
print('Hello, nice to meet you I am the fake chatbot.')
print('Please select the group your question fits in: ')
print('proposal, research, policy, other')
print(' ')
user_input = "proposal"

print('Here are some of our top asked questions for proposals.')
print('Please ask your question if it doesnt fit those.')


Hello, nice to meet you I am the fake chatbot.
Please select the group your question fits in: 
proposal, research, policy, other
 


## Step: embedd as vectors the QA's and tags
My recommednation is to use distill bert

So my thought here is we can make the identical database as the other one just in vector representation. We only vector represent questions and tags since its all connected we can access both databases and get the answer based on the vectorized question... 
I am not sure how the updating would work tho. Would the vectorized database update each time a new entry gets placed in the new database???

In [ ]:
#Installations that may be necessary
#pip install transformers
#pip install torch

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import sqlite3

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert text to BERT embeddings
def get_bert_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    # Use the [CLS] token embedding as the representation of the entire sequence
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Connect to database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Get QA entries from the database
cursor.execute("SELECT question, answer, tag FROM qa_table")
qa_entries = cursor.fetchall()

# Convert QA entries to vectors using BERT embeddings
vectors = []
for question, answer in qa_entries:
    question_vector = get_bert_embedding(question)
    answer_vector = get_bert_embedding(answer)
    tag_vector = get_bert_embedding(tag)
    vectors.append((question_vector, answer_vector, tag_vector))

# Now 'vectors' contains the BERT embeddings for each QA pair
# Now we can use for comparison and similarity scores

# Close connection
conn.close()


## Step 5: Move the vectors to a database
This is so we do not have to reconvert them

In [ ]:
#Installations which may be necessary
#pip install mysql-connector-python

In [ ]:
#Imports
#import torch
#from transformers import BertTokenizer, BertModel
#import mysql.connector
#import numpy as np

In [ ]:
#Code to move them to a database
'''
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert text to BERT embeddings
def get_bert_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    # Use the [CLS] token embedding as the representation of the entire sequence
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Connect to your source database (MySQL)
source_conn = mysql.connector.connect(
    host='your_source_host',
    user='your_username',
    password='your_password',
    database='your_source_database'
)
source_cursor = source_conn.cursor()

# Fetch QA entries from the source database
source_cursor.execute("SELECT question, answer FROM qa_table")
qa_entries = source_cursor.fetchall()

# Convert QA entries to vectors using BERT embeddings
vectors = []
for question, answer in qa_entries:
    question_vector = get_bert_embedding(question)
    answer_vector = get_bert_embedding(answer)
    vectors.append((question_vector, answer_vector))

# Close the source database connection
source_conn.close()

# Connect to your destination database (MySQL)
dest_conn = mysql.connector.connect(
    host='your_dest_host',
    user='your_username',
    password='your_password',
    database='your_dest_database'
)
dest_cursor = dest_conn.cursor()

# Create a table to store vectors in the destination database
dest_cursor.execute('''
    #CREATE TABLE IF NOT EXISTS vector_table (
    #    question_vector BLOB,
    #    answer_vector BLOB
    #)
''')

# Insert vectors into the destination database
for question_vector, answer_vector in vectors:
    dest_cursor.execute("INSERT INTO vector_table (question_vector, answer_vector) VALUES (%s, %s)",
                        (question_vector.tobytes(), answer_vector.tobytes()))

# Commit changes and close the destination database connection
dest_conn.commit()
dest_conn.close()
'''

## Step 6: User entry